# pandas与模型代码的接口

In [1]:
# pandas与其他分析库通常是靠NumPy的数组联系起来的
# 将DataFrame转换为NumPy数组可以使用.value属性
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

In [4]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [5]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [6]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [8]:
# 转换回DataFrame，可以传递一个二维ndarray，可以带有列名
df2 = pd.DataFrame(data.values, columns = ['one', 'two', 'three'])

In [9]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


当数据是均匀的时候使用.value属性（如全是数值类型），如果数据是不均匀的，结果是Python对象的ndarray。

In [10]:
df3 = data.copy()

In [11]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [12]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [13]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [14]:
# 对于只想使用列的子集的情况，可以使用loc，用value作索引
model_cols = ['x0', 'x1']

In [15]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [16]:
# 假设数据集中有一个非数值列
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                 categories = ['a', 'b'])

In [17]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [18]:
# 若想替换category为虚变量，可以创建虚变量，删除category列，然后添加到结果
dummies = pd.get_dummies(data.category, prefix = 'category')

In [19]:
data_with_dummies = data.drop('category', axis = 1).join(dummies)

In [20]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


# 用Pasty创建模型描述

In [21]:
data  = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

In [22]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [24]:
import patsy

In [25]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [26]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [27]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [ ]:
# 这些Patsy的DesignerMatrix实例是NumPy的ndarray，带有附加元数据